In [ ]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

glueContext = GlueContext(SparkContext.getOrCreate())

In [ ]:
#show data1
data1 = glueContext.create_dynamic_frame.from_catalog(database = "data1", table_name="newdataset1_csv").toDF()
print ("Records: ", data1.count())
data1.show()

In [ ]:
#show data2
data2 = glueContext.create_dynamic_frame.from_catalog(database = "data1", table_name="dataset2_csv").toDF()
print ("Records: ", data2.count())
data2.show()

In [ ]:
#merge datasets, adding an extra column indicating the dataset source
from pyspark.sql.functions import lit
datasets = data1.withColumn("source",lit("data1")).unionAll(data2.withColumn("source",lit("data2")))
print ("combined dataset length: ", datasets.count())
datasets.show()

In [ ]:
#data output
datasets.coalesce(1).write.json("s3://usecase1-find-matches/clean/data_clean.json",mode = "overwrite")

In [ ]:
#view labels
labels = glueContext.create_dynamic_frame.from_catalog(database = "data1", table_name="labels_50_csv").toDF()
print ("labels length: ", labels.count())
labels.show()

In [ ]:
#quick preview of the output
output = glueContext.create_dynamic_frame.from_catalog(database = "data1", table_name="output").toDF()
print ("output dataset length: ", output.count())
output.show()

In [ ]:
from pyspark.sql.functions import col

output.sort(col("match_id")).show()

In [ ]:
output.groupby(col("match_id")).count().where(col("count")> 1).show()

In [ ]:
output.filter(output.match_id==226).show()